In [1]:
# Library to read in json files
import urllib as u
from urllib.request import urlopen
import json

# Pandas
import pandas as pd
from pandas_datareader import data

# Library used for itemgetter sort
import operator

# Regex
import re as re

# Progess Bar
from tqdm import tqdm

Parameters to be set for data extraction. This will allow possible future re-use of the notebook

In [2]:
# Set the following Parameters for Data Extraction
api_key='64d94994afb9689c49b660ebe9dee39c'
filename_indicators = 'indicators_reduced.txt'
annual_info = '2020'
price_b_date = '2020-01-01' # No need to set if flag_return is N
price_e_date = '2020-12-31' # No need to set if flag_return is N
filename_data = 'DATA_RAW_2020'
flag_return = 'N'
flag_random = 'Y'

Below five functions that will be used to scrape the Financial Modelling Prep json files and to read the prices variations from Yahoo.

In [3]:
# Function to read in json data from FMP
def get_json_data(url):
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

In [4]:
# Function to get sector of a Stock from its profile json file
def find_sector(json):
    
    for k, v in a.items():
        if k == 'symbol':
            symbol=v

    for k, v in a.items():
        if k == 'profile':
            profile=v

    for k, v in profile.items():
        if k == 'sector':
            sector=v
    return(symbol, sector)

In [5]:
# Function to get Financial Indicators from json file
def find_fin_indicators(json, indicator):
    cat = ['financials','growth','metrics']
    
    for k, v in json.items():
        if k == 'symbol':
            symbol = v

    for k, v in json.items():
        if k in cat:
            category = v

    indicators_annual = [[(k,v) for k,v in d.items()] for d in category if d['date'][:4] == annual_info] # Generator creates a list of lists
    indicators_annual_list = [item for sublist in indicators_annual for item in sublist] # Creating a flat list

    a = []

    for tup in (x for x in indicators_annual_list if x[0] in indicator):
        a.append((symbol,) + tup)
    return a

In [6]:
# Function to get Ratios from json file
def find_ratios_indicators(json, indicator):
    global ratios, symbol
    
    for k in json.items():
        symbol = json['symbol']
    
    for k in json.items():
        ratios = json['ratios']
            
    
    ratios_annual = [d for d in ratios if d['date'][:4] == annual_info]
    
    unpack1 = ratios_annual[0]['investmentValuationRatios']
    unpack2 = ratios_annual[0]['profitabilityIndicatorRatios']
    unpack3 = ratios_annual[0]['operatingPerformanceRatios']
    unpack4 = ratios_annual[0]['liquidityMeasurementRatios']
    unpack5 = ratios_annual[0]['debtRatios']
    unpack6 = ratios_annual[0]['cashFlowIndicatorRatios']
    
    unpack = unpack1
    unpack.update(unpack2)
    unpack.update(unpack3)
    unpack.update(unpack4)
    unpack.update(unpack5)
    unpack.update(unpack6)
    
    full_set = []
    reduced_set =[]

    for row in unpack.items():
            full_set.append(row)
    
    for tup in (x for x in full_set if x[0] in indicator):
        reduced_set.append((symbol,) +tup)
    
    return reduced_set

In [7]:
# Function to get the stock's price variation
def get_price_var(symbol):

    prices = data.DataReader(symbol, 'yahoo', price_b_date, price_e_date)['Adj Close']

    end = prices.index[-1]
    start = prices.index[0]

    price_var = ((prices[end] / prices[start]) - 1) * 100
    
    return symbol, price_var

The following part of the Notebook leverages on Regex to find the urls of the Json Files within a txt file

In [8]:
# Leverage on Regex functionalities to retrieve FMP urls
with open('urls.txt') as f:
    urls = f.readlines()

'''List of Stocks'''
for i in urls:
    if len(re.findall('https.+list',i)) == 0:
        continue
    else: 
        list_of_stocks = str(re.findall('https.+list',i)[0])

'''Profiles'''
for i in urls:
    if len(re.findall('https.+profile\W',i)) == 0:
        continue
    else: 
        profiles = str(re.findall('https.+profile\W',i)[0])
        
'''Income Statement'''
for i in urls:
    if len(re.findall('https.+income.\w+\W',i)) == 0:
        continue
    else: 
        inc_stat = str(re.findall('https.+income.\w+\W',i)[0])
        
'''Cash Flow Statement'''
for i in urls:
    if len(re.findall('https.+cash-flow.\w+\W',i)) == 0:
        continue
    else: 
        cf_stat = str(re.findall('https.+cash-flow.\w+\W',i)[0])
        
'''Balance Sheet'''
for i in urls:
    if len(re.findall('https.+balance-sheet.\w+\W',i)) == 0:
        continue
    else: 
        bs_stat = str(re.findall('https.+balance-sheet.\w+\W',i)[0])
        
'''Financial Statment Growth'''
for i in urls:
    if len(re.findall('https.+growth\W',i)) == 0:
        continue
    else: 
        fs_growth = str(re.findall('https.+growth\W',i)[0])
        
'''Key Metrics'''
for i in urls:
    if len(re.findall('https.+metrics\W',i)) == 0:
        continue
    else: 
        key_metrics = str(re.findall('https.+metrics\W',i)[0])
        
'''Financial Ratios'''
for i in urls:
    if len(re.findall('https.+ratios\W',i)) == 0:
        continue
    else: 
        fin_ratios = str(re.findall('https.+ratios\W',i)[0])

In [9]:
# Full list of available stocks from FMP
url = list_of_stocks + '?apikey=' + api_key
ticks_json = get_json_data(url)

stocks = pd.DataFrame(ticks_json)
stocks = stocks[['symbol','name','exchange']]
stocks = stocks.dropna(how='any',subset=['symbol','name','exchange'])

In [11]:
# Limit the analysis to stocks of the Nasdaq Global
stocks_nyse = stocks[(stocks['exchange'] == 'Nasdaq Global Select') | (stocks['exchange'] == 'New York Stock Exchange')]

if flag_random =='Y':
    stocks_nyse = stocks_nyse.sample(100)
else:
    pass
    
stocks_nyse.reset_index(drop=True, inplace=True)
tickers_nyse = stocks_nyse['symbol'].tolist()

In [12]:
# Get sectors from company profiles
tickers_nyse_sector = []

for tick in tqdm(tickers_nyse):
    url = profiles + tick + '?apikey=' + api_key 
    a = get_json_data(url)
    tickers_nyse_sector.append(find_sector(a))

100%|██████████| 100/100 [01:01<00:00,  1.61it/s]


In [13]:
tickers_nyse_sector = pd.DataFrame(tickers_nyse_sector)
tickers_nyse_sector.columns = ['symbol', 'sector']

stocks_nyse = stocks_nyse.merge(tickers_nyse_sector)

In [14]:
# Load financial indicators from the provided .txt file
indicators = []
filename = filename_indicators
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

In [15]:
# Get all annual indicators
inc_statement_indicators = []
cash_flow_indicators = []
bs_indicators = []
fs_growth_indicators = []
metrics_indicators = []
ratios_indicators = []

for tick in tqdm(tickers_nyse):
    try:
        url0 = inc_stat + tick + '?limit=120&apikey=' + api_key
        url1 = cf_stat + tick + '?limit=120&apikey=' + api_key
        url2 = bs_stat + tick + '?limit=120&apikey=' + api_key
        url3 = fs_growth + tick + '?limit=120&apikey=' + api_key
        url4 = key_metrics + tick + '?limit=40&apikey=' + api_key
        url5 = fin_ratios + tick + '?limit=40&apikey=' + api_key
        
        a0 = get_json_data(url0)
        a1 = get_json_data(url1)
        a2 = get_json_data(url2)
        a3 = get_json_data(url3)
        a4 = get_json_data(url4)
        a5 = get_json_data(url5)
    except:
        pass
    
    try:
        inc_statement_indicators.append(find_fin_indicators(a0,indicators))
        cash_flow_indicators.append(find_fin_indicators(a1,indicators))
        bs_indicators.append(find_fin_indicators(a2,indicators))
        fs_growth_indicators.append(find_fin_indicators(a3,indicators))
        metrics_indicators.append(find_fin_indicators(a4,indicators))
        ratios_indicators.append(find_ratios_indicators(a5,indicators))
    except:
        pass
    
    fin_indicators = inc_statement_indicators + cash_flow_indicators + bs_indicators + fs_growth_indicators + metrics_indicators + ratios_indicators
    fin_indicators_list = [item for sublist in fin_indicators for item in sublist]
    fin_indicators_list.sort(key = operator.itemgetter(0))

100%|██████████| 100/100 [06:49<00:00,  4.09s/it]


In [16]:
# Transform the list of indicators in a DataFrame
df_ind = pd.DataFrame(indicators)
df_ind.columns = ['variable']

In [17]:
# Dataframe by symbol whose indicators were found 
temp = []
df = df_ind

tickers_found = []

for j in fin_indicators_list:
    tickers_found.append(j[0])
    
tickers_found = pd.DataFrame(tickers_found)
tickers_found.columns = ['symbol']
tickers_found = tickers_found.drop_duplicates()
tickers_found.set_index('symbol')
tickers_found = tickers_found['symbol'].to_list()

for i in tqdm(tickers_found):
    for j in fin_indicators_list:
        if j[0] == i:
            temp.append(j[2])
            df_temp = pd.DataFrame(temp)
            df_temp.columns = [j[0]]
    df = df.join(df_temp)
    del df_temp
    temp.clear()

100%|██████████| 66/66 [00:04<00:00, 14.66it/s]


In [18]:
# Set Symbol as Index in Stocks List
stocks_nyse_labelled = stocks_nyse.set_index('symbol')

In [19]:
# Set symbol as Index in Dataframe with Indicators
df_labelled = df.set_index('variable')
df_labelled_trasposed = df_labelled.T
df_labelled_trasposed = df_labelled_trasposed.reset_index()
data_final = df_labelled_trasposed.rename(columns={'index': 'symbol'})
data_final = data_final.set_index('symbol')
data_final = data_final.merge(stocks_nyse_labelled['sector'], left_on=['symbol'], right_on=['symbol'])

In [20]:
# Get returns and save results in a DataFrame
if flag_return =='Y':
    returns = []

    for i in tqdm(tickers_nyse):
        try:
            returns.append(get_price_var(i))
        except:
            pass

    df_returns = pd.DataFrame(returns)
    df_returns.columns = ['symbol','return']
    df_returns = df_returns.set_index('symbol')
else:
    pass

100%|██████████| 100/100 [02:31<00:00,  1.52s/it]


In [21]:
# DataFrame merge with Returns and write final data into .csv
if flag_return =='Y':
    data_raw = data_final.merge(df_returns, left_on=['symbol'], right_on=['symbol'])
else:
    data_raw = data_final 

data_raw.to_csv(filename_data + '.csv')